# PyTorch Autograd

主要参考以下资料，在PyTorch 背景下了解自动微分的基本概念。

- [PyTorch Autograd](https://towardsdatascience.com/pytorch-autograd-understanding-the-heart-of-pytorchs-magic-2686cd94ec95)
- [CSC321 Lecture 10: Automatic Differentiation](https://www.cs.toronto.edu/~rgrosse/courses/csc321_2018/slides/lec10.pdf)

承认吧，就大型神经网络而言，我们都不擅长微积分。通过显式求解数学方程式来计算此类大型复合函数的梯度是不切实际的，尤其是这些曲线存在于高维中并且无法理解。

>要处理14维空间中的超平面，请可视化3D空间并大声对自己说“十四”。每个人都做得到-Geoffrey Hinton

这就是PyTorch的autograd出现的地方。它抽象了复杂的数学，并帮助我们“神奇地”仅用几行代码就可以计算出高维曲线的梯度。本文就简单记录下autograd的这种魔力。

## PyTorch基础

在继续之前，我们需要了解一些基本的PyTorch概念。

- **Tensors张量**：简单来说，就是PyTorch中的n维数组。张量支持一些其他增强功能，使其具有独特性。除CPU外，还可以在GPU上加载它们以进行更快的计算。设置.requires_grad = True后，它们开始形成一个ackward graph，该图使用称为动态计算图（DCG）的东西来跟踪对其应用的每个操作，以计算梯度（后文中进一步解释）。

In [1]:
import torch
import numpy as np

x = torch.randn(2, 2, requires_grad = True)

# From numpy
x = np.array([1., 2., 3.]) #Only Tensors of floating point dtype can require gradients
x = torch.from_numpy(x)
# Now enable gradient 
x.requires_grad_(True)
# _ above makes the change in-place (its a common pytorch thing)

tensor([1., 2., 3.], dtype=torch.float64, requires_grad=True)

注意：根据PyTorch的设计，只能为浮点张量计算梯度，这就是为什么在将其设为启用梯度的PyTorch张量之前创建了一个float类型的numpy数组的原因。

- **Autograd**：该类是计算导数的引擎（更精确地说，是Jacobian-vector product）。它记录了在启用梯度的张量上执行的所有运算，创建了一个称为动态计算图的非循环图。此图的*叶子是输入张量，根是输出张量*。通过从根到叶跟踪图并使用链法则将每个梯度相乘来计算梯度。

## 神经网络与反向传播

神经网络无非就是经过微调（训练）以输出所需结果的复合数学函数。调整或训练是通过一种称为反向传播的出色算法完成的。反向传播用于计算损失相对于输入权重的梯度，以在以后更新权重并最终减少损失。

>在某种程度上，反向传播只是链式规则的奇特名称-Jeremy Howard

创建和训练神经网络涉及以下基本步骤：

1. 定义架构
2. 使用输入数据在体系结构上进行正向传播
3. 计算损失
4. 反向传播以计算每个权重的梯度
5. 使用学习率更新权重

随输入权重的微小变化而得的loss变化称为权重的梯度，并使用反向传播进行计算。然后结合学习率将梯度用于更新权重，以总体上减少损失并训练神经网络。

这是通过迭代的方式完成的。每次迭代，都会计算一些梯度，并建立一个称为计算图的东西来存储这些梯度函数。PyTorch通过构建动态计算图（DCG）来实现。该图是在每次迭代中从头开始构建的，为梯度计算提供了最大的灵活性。例如，对于前向运算（函数）Mul，就有称为MulBackward的后向运算（函数）被动态地集成在后向图中以计算梯度。

## 动态计算图

启用了梯度的张量（变量）与函数（运算）相结合以创建动态计算图。数据流和应用于数据的运算是在运行时定义的，因此可以动态构建计算图。该图由autograd类动态创建。在开始训练之前，不必编码所有可能的路径，所要运行的就是要微分的。

一个用于将两个张量相乘的简单DCG看起来像这样：

![](img/1_jGo_2J9UQeynwG_3olUD4w.png)

图中的每个虚线轮廓框都是一个变量，紫色矩形框是一个运算。

每个变量对象都有几个成员，其中一些是：

- 数据：这是变量保存的数据。x持有一个1x1张量，其值等于1.0，而y持有2.0。z持有两者乘积，即2.0
- require_grad：如果为true，则开始跟踪所有运算历史记录并形成用于梯度计算的后向图。对于任意张量a ，可以按如下方式对其进行就地操作：a.requires_grad_(True)
- grad： grad保存梯度的值。如果requires_grad为False，则将保留None值。即使requires_grad为True，它也将保持None值，除非.backward()从其他某个节点调用了该函数。例如，如果调用out.backward()的一些变量out中包括了X在其计算中，那么x.grad将持有∂out/∂x。
- grad_fn：这是用于计算梯度的后向函数。
- is_leaf：如果满足以下条件，则节点为叶节点：
    1. 它是通过诸如x = torch.tensor(1.0)或x = torch.randn(1, 1)（基本上是本文开头讨论的所有张量初始化方法）之类的函数显式初始化的。
    2. 它是在对所有具有requires_grad = False的张量进行运算后创建的.
    3. 它是通过在某个张量上调用.detach()方法创建的。

在调用backward()时，仅对requires_grad和is_leaf同时为True的节点填充梯度。梯度属于调用.backward()的输出节点关于其他叶节点的。

一设置requires_grad = True，PyTorch就将开始跟踪运算并在每个步骤中存储梯度函数，如下所示：

![](img/1_viCEZbSODfA8ZA4ECPwHxQ.png)

在PyTorch中生成上述图的代码是：

In [2]:
import torch

# Creating the graph
x = torch.tensor(1.0, requires_grad = True)
y = torch.tensor(2.0)
z = x * y

# Displaying
for i, name in zip([x, y, z], "xyz"):
    print(f"{name}\ndata: {i.data}\nrequires_grad: {i.requires_grad}\n\
grad: {i.grad}\ngrad_fn: {i.grad_fn}\nis_leaf: {i.is_leaf}\n")

x
data: 1.0
requires_grad: True
grad: None
grad_fn: None
is_leaf: True

y
data: 2.0
requires_grad: False
grad: None
grad_fn: None
is_leaf: True

z
data: 2.0
requires_grad: True
grad: None
grad_fn: <MulBackward0 object at 0x0000020AA9120820>
is_leaf: False



<ipython-input-2-3711e5bd1754>:11: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  grad: {i.grad}\ngrad_fn: {i.grad_fn}\nis_leaf: {i.is_leaf}\n")


要阻止PyTorch跟踪历史记录并形成后向图，可以将代码包装在with torch.no_grad()中，这样就可以在不需要梯度跟踪时使代码运行更快。

In [3]:
import torch
# Creating the graph
x = torch.tensor(1.0, requires_grad = True)
# Check if tracking is enabled
print(x.requires_grad) #True
y = x * 2
print(y.requires_grad) #True

with torch.no_grad():
	# Check if tracking is enabled
	y = x * 2
	print(y.requires_grad) #False

True
True
False


## Backward（）函数

Backward是通过 在后向图中从根张量到到每个叶节点的所有路径 传递参数(默认为1x1单位张量)来计算梯度的函数。然后将计算出的梯度存储在每个叶节点的.grad中。后向图已经在前向传递过程中动态创建。后向函数仅使用已制作的图计算梯度并将其存储在叶节点中。

分析以下代码

In [4]:
import torch
# Creating the graph
x = torch.tensor(1.0, requires_grad = True)
z = x ** 3
z.backward() #Computes the gradient 
print(x.grad.data) #Prints '3' which is dz/dx 

tensor(3.)


需要注意的重要一点是，当z.backward()被调用时，张量会自动作为z.backward(torch.tensor(1.0))传递。torch.tensor(1.0)是外部梯度，用以终止链式法则梯度乘法。将此外部梯度作为输入传递给MulBackward函数，以进一步计算x的梯度。传入的张量的尺寸.backward()必须与要计算其梯度的张量的尺寸相同。例如，如果启用梯度的张量x和y如下所示：

x = torch.tensor([0.0, 2.0, 8.0], requires_grad = True)

y = torch.tensor([5.0 , 1.0 , 7.0], requires_grad = True)

z = x * y

然后，要计算z相对于x或的（1x3张量）y的梯度，需要将外部梯度传递给 z.backward()函数，如下所示：

z.backward(torch.FloatTensor([1.0, 1.0, 1.0])

而z.backward() 会给一个 RuntimeError: grad can be implicitly created only for scalar outputs

传递到后向函数中的张量的作用类似于权重，用于梯度的加权输出。从数学上讲，这是矢量乘以非标量张量的雅可比矩阵，因此backward ，除非需要计算加权输出，否则它几乎应该始终是与调用张量相同的尺寸的单位张量。

>tldr：后向图由autograd类在前向过程中自动动态创建。Backward()只需将其参数传递给已制成的后向图即可计算梯度。

## 数学—雅可比和向量

从数学上讲，autograd类只是一个雅可比矢量积计算引擎。雅可比矩阵，简单说，就是表示两个向量的所有可能的偏导数的矩阵。它是一个向量相对于另一个向量的梯度。

假设启用了PyTorch梯度的张量X为：

$X = [x_1,x_2,\dots,x_n]$（可以认为这是某些机器学习模型的权重）

X进行一些运算以形成向量Y

$Y = f(X)= [y_1,y_2,\dots y_m]$

那么雅可比阵就是：

$$J = \begin{bmatrix}
 \frac{\partial y_1}{\partial{x_1}}  \ ... \ \frac{\partial{y_m}}{\partial{x_1}} \\ 
  \vdots \ \ddots \ \vdots \\
 \frac{\partial{y_1}}{\partial{x_n}}  \ ... \ \frac{\partial{y_m}}{\partial{x_n}} \\
\end{bmatrix} $$

然后，使用Y来计算标量损失l。假设向量v是标量损失l相对于向量Y的梯度，如下所示

$$v=(\frac{\partial l}{\partial y_1} \dots \frac{\partial l}{\partial y_m})^T$$

向量v称为 grad_tensor，并作为参数传递给backward()函数

为了获得损失l相对于权重X的梯度，将Jacobian矩阵J与向量v相乘

$$J^T \cdot v = \begin{bmatrix}
 \frac{\partial y_1}{\partial{x_1}}  \ ... \ \frac{\partial{y_m}}{\partial{x_1}} \\ 
  \vdots \ \ddots \ \vdots \\
 \frac{\partial{y_1}}{\partial{x_n}}  \ ... \ \frac{\partial{y_m}}{\partial{x_n}} \\
\end{bmatrix} \begin{bmatrix}
 \frac{\partial l}{\partial{y_1}} \\ 
 \vdots \\ 
 \frac{\partial{l}}{\partial{y_m} }
\end{bmatrix} = \begin{bmatrix}
 \frac{\partial{l}}{\partial{x_1}} \\ 
 \vdots \\ 
 \frac{\partial{l}}{\partial{x_m}} 
\end{bmatrix}$$

按一步反向传播写，可以表示为:

$$\bar X=J^T \bar Y$$

即Y对X的雅可比阵乘以，对Y的梯度，就是对X的梯度。

在反向传播过程中，PyTorch不会显式地专门先构造好整个Jacobian。而是直接计算上面的乘积，即Vector-Jacobian Products（VJP），这样更简单，更高效。

这种计算雅可比矩阵并将其与向量v相乘的方法使PyTorch甚至可以轻松地为非标量输出提供梯度，具体细节可以先不管，总之，backward()，还有其他像Autograd中的defvjp()算的都是这东西。